## Module 6: Model Selection & Regularization Techniques for Regression

### Step 0

Load the appropriate libraries and bring in the data. Note that we have to run a script to get the [California Housing dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html) to match as it is in scikit-learn. We cannot pull it directly from scikit-learn since CodeGrade cannot access the internet.

In [1]:
# CodeGrade step0

from sklearn.datasets import fetch_california_housing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
import os
import tarfile
import joblib # Import joblib directly
from sklearn.datasets._base import _pkl_filepath, get_data_home
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


archive_path = "cal_housing.tgz" # change the path if it's not in the current directory
data_home = get_data_home(data_home=None) # change data_home if you are not using ~/scikit_learn_data
if not os.path.exists(data_home):
    os.makedirs(data_home)
filepath = _pkl_filepath(data_home, 'cal_housing.pkz')

with tarfile.open(mode="r:gz", name=archive_path) as f:
    cal_housing = np.loadtxt(
        f.extractfile('CaliforniaHousing/cal_housing.data'),
        delimiter=',')
    # Columns are not in the same order compared to the previous
    # URL resource on lib.stat.cmu.edu
    columns_index = [8, 7, 2, 3, 4, 5, 6, 1, 0]
    cal_housing = cal_housing[:, columns_index]

    joblib.dump(cal_housing, filepath, compress=6) # Now using the directly imported joblib

# Load dataset
california = fetch_california_housing(as_frame=True)
data = california.data
data['MedianHouseValue'] = california.target

# Define predictors and response variable
X = data[['MedInc', 'AveRooms', 'AveOccup']]  # Select predictors
y = data['MedianHouseValue']  # Response variable

Print the basic information of the data using `.info()` and `.describe`.

In [2]:
# Display dataset structure
print(data.head())
print(data.info())
print(data.describe())

   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  MedianHouseValue  
0    -122.23             4.526  
1    -122.22             3.585  
2    -122.24             3.521  
3    -122.25             3.413  
4    -122.25             3.422  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MedInc            20640 non-null  float64
 1   HouseAge          20640 non-null  float64
 2   AveRooms          20640 non-null  float64
 3

### Step 1

Add a constant to `X`, calling it `X_const`.

Now let `full_model` be the OLS model of `y` and `X_const`.

Rounding to the nearest whole number, return the full model's aic and bic, separated by a comma.

In [11]:
# CodeGrade step1
# add constant to X
X = sm.add_constant(X)

# fit the model
full_model = sm.OLS(y, X).fit()

# return AIC and BIC, rounded to the nearest whole number
print(f"Full Model AIC: {round(full_model.aic)},")
print(f"Full Model BIC: {round(full_model.bic)}")

Full Model AIC: 50962,
Full Model BIC: 50994


### Step 2


Let `subset1` be the OLS model fit with `MedInc` and `AveRooms` and `subset2` be the OLS model fit with `MedInc` and `AveOccup`.

Again rounding to the nearest whole number, give the AIC and BIC for the first subset and then the the same two information criteria for the second subset. These four values should be separated by commas.

In [23]:
# CodeGrade step2
# subset models

# model with MedInc and AveRooms
subset1 = sm.add_constant(data[['MedInc', 'AveRooms']])

# model with MedInc and AveOccup
subset2 = sm.add_constant(data[['MedInc', 'AveOccup']])

# fit models
model1 = sm.OLS(y, subset1).fit()
model2 = sm.OLS(y, subset2).fit()

# return AIC and BIC
print(f"Model 1 AIC: {round(model1.aic)}, BIC: {round(model1.bic)},")
print(f"Model 2 AIC: {round(model2.aic)}, BIC: {round(model2.bic)},")

Model 1 AIC: 51016, BIC: 51040,
Model 2 AIC: 51199, BIC: 51222,


Run the below code without change for the ridge model.

In [14]:
# CodeGrade step0

seed = 42

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ridge regression
ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)

Ridge()

### Step 3

Define `ridge_pred` as the prediction of `X_test` and `ridge_mse` as the mean squared error of `y_test` and `ridge_pred`.

Return `ridge_mse`, rounded to four decimal places.

In [16]:
# CodeGrade step3
# predict and calculate MSE
ridge_pred = ridge.predict(X_test)
ridge_mse = mean_squared_error(y_test, ridge_pred)

# return ridge_mse, rounded to 4 decimal places
print(f"Ridge MSE: {round(ridge_mse, 4)}")

Ridge MSE: 0.7007


### Step 4

Return `ridge.coef_`.

In [17]:
# CodeGrade step4

print(f"Ridge coefficients: {ridge.coef_}")

Ridge coefficients: [ 0.          0.43687436 -0.04042693 -0.00382598]


Run the below code without change for the lasso model.

In [18]:
# CodeGrade step0

# Lasso regression
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

### Step 5


Define `lasso_pred` as the prediction of `X_test` and `lasso_mse` as the mean squared error of `y_test` and `lasso_pred`.

Return `lasso_mse`, rounded to four decimal places.

In [19]:
# CodeGrade step5
# define lasso-pred and find lasso_mse based on y_test and lasso_pre
lasso_pred = lasso.predict(X_test)
lasso_mse = mean_squared_error(y_test, lasso_pred)

# return lasso_mse rounded to 4 decimal places
print(f"Lasso MSE: {lasso_mse:.4f}")

Lasso MSE: 0.7047


Print all of the results.

In [22]:
# Print all model results

# full model AIC and BIC
print(f"Full Model AIC: {round(full_model.aic)}, Full Model BIC: {round(full_model.bic)}")

# Model 1 AIC and BIC
print(f"Model 1 AIC: {round(model1.aic)}, BIC: {round(model1.bic)},")

# model 2 AIC and BIC
print(f"Model 2 AIC: {round(model2.aic)}, BIC: {round(model2.bic)}")

# Ridge MSE and coefficients
print(f"Ridge MSE: {round(ridge_mse, 4)}")
print(f"Ridge coefficients: {ridge.coef_}")

# Lasso MSE and coefficients
print(f"Lasso MSE: {lasso_mse:.4f}")
print(f"Lasso coefficients: {lasso.coef_}")

Full Model AIC: 50962, Full Model BIC: 50994
Model 1 AIC: 51016, BIC: 51040,
Model 2 AIC: 51199, BIC: 51222
Ridge MSE: 0.7007
Ridge coefficients: [ 0.          0.43687436 -0.04042693 -0.00382598]
Lasso MSE: 0.7047
Lasso coefficients: [ 0.          0.39731277 -0.01225582 -0.00290792]
